In [ ]:
import numpy as np
import glob
import os 
from ultralytics import YOLO
import yolo_util as yutl
import numpy as np
import pandas as pd
import cv2
import argparse

def stream_segment(cap, start_frame, end_frame,label,raceid,FRV8,FRV8data, fr=25,windows=[32],SEQIMG_PATH="IMGSEQ_NOT_15S",FRV8PRELOAD=[],modelName ="FRV_ESDSEQ",saveinterval=15):
    print(f"Stream Segment with preload FRV8: {label}"  , start_frame-(fr*max(windows)) ,"Extract between ")
    frame_count = int(start_frame)
    print(f"SKIP TO {frame_count}")
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_count)
    FRV8stack =[]
    frame_seq_buffer=[] 
    while cap.isOpened():
        ret, frame = cap.read()
        if frame_count > end_frame:
            break
        if ret:
            frame_count += 1
            # if frame_count <  start_frame-(fr*max(windows+1)):
            #     continue
            if frame_count % 5 == 0:
                print(frame_count)
                # frame= cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
                # if len(FRV8PRELOAD) > 0:
                #     FRV8now = FRV8PRELOAD[FRV8PRELOAD["filename"]==frame_count].values
                #     # print(FRV8now)
                # else:
                #     FRV8stack, FRV8now = yutl.getDet(frameinput=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB),frame_count=frame_count,
                #                     model=FRV8,names=FRV8data,
                #                     cumulative_det=FRV8stack)
                # ---------------------------------------------------------------------------------------------------------
                frame_seq_buffer,grids = yutl.createSeqGeneral(seq_frame=frame,drawobjs=[],modelName=modelName,
                                                        list_seq_frame=frame_seq_buffer,
                                                        frame_counter=frame_count, windows=windows)
                if len(frame_seq_buffer) > 9:
                    frame_seq_buffer = frame_seq_buffer[1:]#Move Window 1+ frame and remove last
                    # SEQstack, SEQnow = yutl.getDet(frameinput=grids,frame_count=frame_count,
                    #             model=SEQ,names=SEQdata,
                    #             cumulative_det=SEQstack,batchtag=windows)
                    wx = 0
                    for grid in grids:

                        if frame_count % saveinterval == 0:
                            image_np = np.array(grid)
                            SAVEPATH =f"{SEQIMG_PATH}/SQ{windows[wx]}/{label}"
                            os.makedirs(f"{SAVEPATH}",exist_ok=True)
                            cf = int(frame_count)
                            grid.save(f"{SAVEPATH}/{raceid}_{cf}_SQ{windows[wx]}_{label}.jpg")
                            # Convert RGB to BGR
                            # image_bgr = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)
                            # cv2.imshow('SEQ_MONITOR', image_bgr)
                        wx = wx +1
        else:
            # Skip to the next frame if read fails
            print("Exit no more frames")
            return 0
        
print("SSSSSSSSSSSSSSSSSSSSSS")
# parser = argparse.ArgumentParser(description='Process some CSV data.')
# parser.add_argument('csv', type=str, help='The path to the CSV file',default="datapoints.csv")
# parser.add_argument('gpu', type=str, help='The gpu index' , default=0)
# parser.add_argument('raceid', type=str, help='The raceid index' , default=0)
# args = parser.parse_args()

# SEQ.tocuda(args.gpu)
# ESD.tocuda(args.gpu)
# FRV8.tocuda(args.gpu)
# print("Runngon on GPU ",args.gpu)
from concurrent.futures import ThreadPoolExecutor
def process_race(raceid, events, network_path, FRV8,FRV8data,SEQIMG_PATH="FRV_ESDSEQ",FRV8PRELOAD=[],windows=[32],modelName="FRV_ESDSEQ"):
    # try:
    onerace = events[events["raceid"] == raceid]
    print(f"{raceid} -- ",len(onerace))
    trk = str(raceid)[-5:-2]

    VID_PATH = f"{network_path}/media/amir/SSD_B/FR/FRV"
    video_path = f"{VID_PATH}/{trk}/{raceid}.mp4"
    print("Loaded Video from : ",video_path)
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video stream or file")
        cap.release()
        
    # try:
    print("EXTRACT list : ",len(onerace),onerace)
    for index, row in onerace.iterrows():
        start_frame = int(row['start'])
        end_frame = int(row['end'])
        label = row['label']
        print(f"Streaming segment for race {row['raceid']} ({row['label']}) from {start_frame} to {end_frame} seconds.")
        print("WOKRING ON : ",cap, start_frame, end_frame,label,raceid,SEQIMG_PATH)
        stream_segment(cap, start_frame, end_frame,label,raceid,FRV8,FRV8data, fr=25,windows=windows,SEQIMG_PATH=SEQIMG_PATH,FRV8PRELOAD=[],modelName =modelName)

    # except:

    #     pass
    if cap.isOpened():
        cap.release()
    # except:
    #     return 0

In [39]:
try:
    network_path = "/run/user/1000/gvfs/sftp:host=192.168.1.52"
    ESD_PATH =f"{network_path}/media/amir/SSD_B/YOLO/MODELS/YOLOv8n_ESD159"
    ESD , ESDdata = yutl.loadYOLO(ESD_PATH,0)
except:
    network_path = ""
    ESD_PATH =f"{network_path}/media/amir/SSD_B/YOLO/MODELS/YOLOv8n_ESD159"
    ESD , ESDdata = yutl.loadYOLO(ESD_PATH,0)
FRV8_PATH =f"{network_path}/media/amir/SSD_B/YOLO/MODELS/FRV8"
FRV8 , FRV8data = yutl.loadYOLO(FRV8_PATH,0)
# SEQ_PATH =f"{network_path}/media/amir/SSD_B/YOLO/MODELS/SEQ_X"
# SEQ , SEQdata = yutl.loadYOLO(SEQ_PATH)
video_dir = f"{network_path}/media/amir/SSD_B/FR/FRV/*/"

In [42]:
DIRECTIONNAME = "ESDSEQ"
events = pd.read_csv(f"ESD_SEQ.csv")[["raceid", "label", "start", "end"]]
w = events[(events["start"] > 0)&(events["label"] != "NOT")]
worktodo = w[(w["label"] == "T001FRV")]
worktodo

,raceid,label,start,end
0,2021041700105,T001FRV,16020,16255
1,2023010100106,T001FRV,8425,9155
3,2023090900105,T001FRV,6565,6780
6,2021121500102,T001FRV,7855,8145
7,2023010300103,T001FRV,7575,7775
...,...,...,...,...
3578,2023012900103,T001FRV,7085,7800
3579,2022120300101,T001FRV,13745,13885
3580,2022120300101,T001FRV,14070,14175
3581,2024012300103,T001FRV,6725,7300


In [43]:

SEQIMG_PATH = f"{DIRECTIONNAME}_DATA_TEST"
events = worktodo#[:20]
with ThreadPoolExecutor(max_workers=15) as executor:
    for raceid in events["raceid"].unique():
        dst = f"{SEQIMG_PATH}/3X3/{raceid}"
        if os.path.exists(dst):
            print("Skeeep")
        else:
            # os.makedirs(dst,exist_ok=True)
            CSVnetwork_path = "/run/user/1000/gvfs/sftp:host=192.168.1.3"
            CSVnetwork_path = "NOTHING"
            preloadpath = f"{CSVnetwork_path}/media/infer/SSDC/computer_vision/extractions/FRV/FRV_OBJ_8DHORSE_000/{raceid}.csv"
            if os.path.exists(preloadpath):
                preloaddf = pd.read_csv(preloadpath)
                print("Preload Dets ",len(preloaddf) ,"Frames", int(preloaddf["filename"].max()))
                # print("Unique Frame",len(preloaddf["filename"].unique()))
                # min = int(preloaddf["filename"].min())
                # max = int(preloaddf["filename"].max())
                # print("Min Max",min,max)
                # frames = max - min
                # print("Frames",frames/5,"Unique Frames",len(preloaddf["filename"].unique()))
            else:
                preloaddf = []
            
            executor.submit(process_race, raceid, worktodo, network_path, FRV8, FRV8data, SEQIMG_PATH=SEQIMG_PATH, FRV8PRELOAD=preloaddf,windows=[3],modelName="FRV_ESDSEQ3")

2021041700105 --  1
Loaded Video from :  /media/amir/SSD_B/FR/FRV/001/2021041700105.mp4
2023010100106 --  1
Loaded Video from :  /media/amir/SSD_B/FR/FRV/001/2023010100106.mp4
2023090900105 --  1
Loaded Video from :  /media/amir/SSD_B/FR/FRV/001/2023090900105.mp4
2021121500102 --  1
Loaded Video from :  /media/amir/SSD_B/FR/FRV/001/2021121500102.mp4
2023010300103 --  1
Loaded Video from :  /media/amir/SSD_B/FR/FRV/001/2023010300103.mp4
2024010900101 --  3
Loaded Video from :  /media/amir/SSD_B/FR/FRV/001/2024010900101.mp4
2021112700108 --  6
Loaded Video from :  /media/amir/SSD_B/FR/FRV/001/2021112700108.mp4
2022122700107 --  1
Loaded Video from :  /media/amir/SSD_B/FR/FRV/001/2022122700107.mp4
2022100400105 --  2
Loaded Video from :  /media/amir/SSD_B/FR/FRV/001/2022100400105.mp4
2023060900105 --  9
Loaded Video from :  /media/amir/SSD_B/FR/FRV/001/2023060900105.mp4
2023021300101 --  2
Loaded Video from :  /media/amir/SSD_B/FR/FRV/001/2023021300101.mp4
2022060900104 --  1
Loaded Video

[ WARN:27@76176.187] global cap.cpp:164 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.10.0) /io/opencv/modules/videoio/src/cap_images.cpp:274: error: (-215:Assertion failed) number < max_number in function 'icvExtractPattern'




9390
EXTRACT list :  5              raceid    label  start    end
514   2021123000103  T001FRV   7955   8185
515   2021123000103  T001FRV   8825   9360
516   2021123000103  T001FRV   6910   7320
517   2021123000103  T001FRV   9700  10215
3314  2021123000103  T001FRV   7630   7780
Streaming segment for race 2021123000103 (T001FRV) from 7955 to 8185 seconds.
WOKRING ON :  < cv2.VideoCapture 0x792070629050> 7955 8185 T001FRV 2021123000103 ESDSEQ_DATA_TEST
Stream Segment with preload FRV8: T001FRV 7880 Extract between 
SKIP TO 7955
9165
25720
7860
12815
11040
8945
13820
9035
6825
12910
8700
25725
9550
7760
12915
9040
9170
11045
6830
9395
7865
8950
13825
12820
7960
25730
8705
9175
9555
6835
13830
8955
9180
7765
25735
7870
11050
9400
7965
9045
9560
12920
6840
8710
12825
9185
7770
8960
13835
12925
9565
6845
25740
9405
7875
9050
11055
7970
Streaming segment for race 2022061000106 (T001FRV) from 11980 to 12750 seconds.
WOKRING ON :  < cv2.VideoCapture 0x79207062a770> 11980 12750 T001FRV 2022061

[ WARN:28@76274.299] global cap.cpp:164 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.10.0) /io/opencv/modules/videoio/src/cap_images.cpp:274: error: (-215:Assertion failed) number < max_number in function 'icvExtractPattern'




9715
6090
8235
15770
6395
8515
10105
7230
13415
8005
10360
7240
6720
15775
6095
8010
8240
6605
16610
7235
2023091700108 --  1
Loaded Video from :  /media/amir/SSD_B/FR/FRV/001/2023091700108.mp4
9720
6725
EXTRACT list :  1             raceid    label  start   end
752  2023091700108  T001FRV   6700  6790
Streaming segment for race 2023091700108 (T001FRV) from 6700 to 6790 seconds.
WOKRING ON :  < cv2.VideoCapture 0x792070774fb0> 6700 6790 T001FRV 2023091700108 ESDSEQ_DATA_TEST
Stream Segment with preload FRV8: T001FRV 6625 Extract between 
SKIP TO 6700
10110
13420
8520
15780
10365
8015
8245
7245
7240
6100
16615
6705
8525
13425
7250
6610
7245
6730
9725
8250
15785
10370
10115
6105
16620
8020
6710
7255
8530
8255
9730
10120
13430
7250
6615
15790
6110
6715
10125
16625
10375
8535
6735
9735
8025
8260
15795
6115
10130
7260
10380
6740
7255
Streaming segment for race 2022022100107 (T001FRV) from 8810 to 9575 seconds.
WOKRING ON :  < cv2.VideoCapture 0x792047854710> 8810 9575 T001FRV 2022022100107 

...

11830
12365
7545
12115
17150
20550
6240
7335
7795
Streaming segment for race 2021112300105 (T001FRV) from 12090 to 12590 seconds.
WOKRING ON :  < cv2.VideoCapture 0x792047854670> 12090 12590 T001FRV 2021112300105 ESDSEQ_DATA_TEST
Stream Segment with preload FRV8: T001FRV 12015 Extract between 
SKIP TO 12090
8100
7225
7550
11395
12095
12120
6245
12370
8990
19115
17155
8105
20555
8105
7340
7555
7800
12125
7230
11400
8995
12100
19120
12130
12375
7560
6250
7345
17160
20560
8110
7235
7805
8110
9000
11405
12105
19125
12135
7810
8115
6255
20565
12380
7350
17165
7565
7240
9005
8115
11410
12110
7570
7815
7245
12140
8120
7355
12115
8120
9010
20570
12385
6260
19130
17170
11415
7575
9015
7820
12145
8125
7360
17175
20575
7250
8125
19135
12390
12120
7365
6265
11420
12150
7255
20580
8130
7580
7825
9020
12125
8130
12395
19140
6270
17180
11425
20585
12155
7830
12130
7585
7370
12400
7260
8135
8135
6275
19145
17185
9025
12160
11430
7590
12135
Streaming segment for race 2021102900101 (T001FRV) from 8450 t

[ WARN:37@76807.321] global cap.cpp:164 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.10.0) /io/opencv/modules/videoio/src/cap_images.cpp:274: error: (-215:Assertion failed) number < max_number in function 'icvExtractPattern'




6140
15635
9420
11735
9015
7190
8920
12585
8065
6735
6215
8435
12635
6155
6145
11740
7195
7550
15640
6740
8070
7200
7555
11745
9425
9020
8440
8925
6150
6160
6220
12590
12640
6745
7560
9025
15645
9430
6165
8930
8445
6155
8075
6225
12645
12595
9030
11750
6750
7205
6160
8450
7565
8080
6170
9435
15650
6230
8935
12600
9035
11755
7210
12650
8455
7570
9440
6165
8085
6175
6235
15655
8940
11760
9040
6755
8460
12605
12655
7575
7215
9045
9445
6240
15660
8945
11765
8465
6180
12660
7580
8090
6760
6170
12610
9050
6185
8950
6245
9450
7220
15665
11770
9055
8470
6190
12615
7585
6765
8095
8955
12665
6175
15670
6250
11775
9455
7225
9060
8100
8475
8960
12620
7590
9460
6255
6195
6180
6770
7230
15675
11780
12670
8965
8480
6185
9065
6260
7235
9465
12625
8105
6200
7595
6775
11785
9070
7240
6265
12630
8970
9470
11790
15680
8485
6190
12675
7600
6270
6205
9475
8110
12635
8975
7245
6780
6195
12680
9075
6210
12640
15685
11795
9480
8490
8980
12685
7605
12645
8985
8115
Streaming segment for race 2024012300103 (T001F

[ WARN:33@76829.508] global cap.cpp:164 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.10.0) /io/opencv/modules/videoio/src/cap_images.cpp:274: error: (-215:Assertion failed) number < max_number in function 'icvExtractPattern'




10740
8780
24685
24585
6980
6390
6465
7545
19985
10745
6705
8620
6950
8785
24590
16680
7550
24690
6985
6395
19990
6215
8625
7555
10750
6710
16685
24695
6470
8790
6400
6955
24595
6475
6990
6405
6220
19995
7560
24600
7055
8630
6715
10755
6410
24700
16690
6960
6480
8795
6225
8635
6995
7565
20000
24705
7060
24605
10760
6415
6720
7000
8800
7570
6485
20005
16695
10765
6230
6965
7005
8640
6420
7575
24610
6725
20010
8805
7065
6490
24710
10770
6970
6235
6425
6495
7010
16700
7580
6730
24615
7070
6240
8645
6975
20015
24715
10775
8810
16705
6430
6500
7015
7075
6735
10780
20020
24720
6435
16710
8650
6505
6245
7020
7080
10785
8815
7585
24620
6980
16715
6440
6250
6740
8655
20025
Streaming segment for race 2023112400101 (T001FRV) from 6285 to 7195 seconds.
WOKRING ON :  < cv2.VideoCapture 0x7920706c8af0> 6285 7195 T001FRV 2023112400101 ESDSEQ_DATA_TEST
Stream Segment with preload FRV8: T001FRV 6210 Extract between 
SKIP TO 6285
24625
7025
10790
24725
7590
7085
6290
8820
6445
6985
8660
16720
20030
6745

In [ ]:
len(events["raceid"].unique())
events["raceid"].unique()

In [ ]:
import os 
rid = 2022052108704
if os.path.exists(f"/media/amir/SSD_B/YOLO/IMGSEQXV4/SQ16/{rid}"):
    print("Skeeep")

In [ ]:
glob.glob("/media/amir/SSD_B/YOLO/IMGSEQXV4/SQ16/2022052108704")

In [26]:
# cap = cv2.VideoCapture("/run/user/1000/gvfs/sftp:host=192.168.1.52/media/amir/SSD_B/FR/FRV/001/2023091700108.mp4")
# cap.set(cv2.CAP_PROP_POS_FRAMES, 6045)
# f= 6045
# while cap.isOpened():
#     ret, frame = cap.read()
#     if ret:
#         f = f + 1
#         if f % 5  == 0:
#             print(f,frame.shape)
#             cv2.imshow('SEQ_MONITOR', frame)
#             # opencv_image = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
#             # Display the image using OpenCV
#             # cv2.imshow("Image", opencv_image)
#             cv2.waitKey(1)
#     else:
#         break

# cv2.destroyAllWindows()